In [ ]:
import sys
sys.path.append("..")

In [ ]:
from src.elevators import *
pd.options.plotting.backend = "plotly"

In [ ]:
# read sum of snapshots/listed/active per day and id
df_all = elevators_per_day()
df_all["listed_f"] = df_all["listed"] / df_all["snapshots"]
df_all["active_f"] = df_all["active"] / df_all["snapshots"]
df_all["inactive_f"] = df_all["inactive"] / df_all["snapshots"]

In [ ]:
# get the defect machines per day
dfs = []
for mark in (0.2, 0.4, 0.6, 0.8, 1.):
    df = df_all[df_all["inactive_f"] >= mark]
    dates = df.index.get_level_values("date").unique().sort_values()
    df_d = pd.DataFrame(
        [df[df.index.get_level_values("date") == d].shape[0] for d in dates],
        index=dates,
        columns=[f"{mark*100:.0f}%"]
    )
    dfs.append(df_d)
df = pd.concat(dfs, axis=1)

In [ ]:
df.plot(
    title="Number of inactive elevators/escaltors per day", 
    labels={
        "variable": "Percentage inactive per day", 
        "value": "Number of inactive machines",
    },
    color_discrete_sequence=["#0d0", "#3b0", "#690", "#860", "#c30"]
)